# Victorian Liquor Licences by location

Data: https://www.data.vic.gov.au/data/dataset/victorian-liquor-licences-by-location

In [12]:
#alcohol environment data for liveability indicators

In [13]:
import pandas as pd
import folium
import geopandas as gpd

In [14]:
locations = pd.read_excel("current_victorian_licences_by_location_april_2019.xlsx", skiprows = 3)

locations.head()

,Licence Num,Licensee,Trading As,Category,Trading Hours,After 11 pm,Maximum Capacity,Address,Suburb,Postcode,...,Longitude,Unnamed: 12,Unnamed: 13,Postal Address,Postal Suburb,Postal Postcode,Council,Region,Metro/Regional,Gaming
0,36139603,RESERVOIR CRICKET CLUB INC,RESERVOIR CRICKET CLUB,Limited Licence,Limited Licence,99,NaN,DONATH RESERVE WEST PAVILION JOHNSON STREET,RESERVOIR,3073.0,...,145.011408,NaN,NaN,DONATH RESERVE WEST PAVILION JOHNSON STREET,RESERVOIR,3073,DAREBIN CITY COUNCIL,Metro North,Metro,N
1,36139687,WODONGA CRICKET CLUB INC,WODONGA CRICKET CLUB,Limited Licence,Limited Licence,99,NaN,GORDON STREET,WODONGA,3690.0,...,146.882671,NaN,NaN,PO BOX 1175,WODONGA,3690,WODONGA CITY COUNCIL,Hume,Regional,N
2,36139695,MADDEN-GREY SIMONE,HAPPY WINE WOMAN,Limited Licence,Limited Licence,99,NaN,U 209 12-14 DICKENS ST,ELWOOD,3184.0,...,144.986657,NaN,NaN,U 209 12-14 DICKENS ST,ELWOOD,3184,PORT PHILLIP CITY COUNCIL,Metro South,Metro,N
3,36139726,TACO BILL SM PTY LTD,TACO BILL MEXICAN CANTINA,Limited Licence,Limited Licence,99,NaN,GROUND FLOOR 375-381 CLARENDON STREET,SOUTH MELBOURNE,3205.0,...,144.962283,NaN,NaN,888 PRINCES HIGHWAY,SPRINGVALE,3171,PORT PHILLIP CITY COUNCIL,Metro South,Metro,N
4,36139768,LANEWAY GREENS PTY LTD,LANEWAY GREENS,Limited Licence,Limited Licence,99,NaN,"LEVEL 13, 564 ST KILDA ROAD",MELBOURNE SOUTH,3004.0,...,144.978855,NaN,NaN,"LEVEL 13, 564 ST KILDA ROAD",MELBOURNE SOUTH,3004,PORT PHILLIP CITY COUNCIL,Metro South,Metro,N


In [15]:
locations.columns

select_cols = ['Trading As', 'Category', 'Trading Hours','Address', 'Suburb', 'Postcode',

       'Latitude', 'Longitude', 'Postal Postcode', 'Council', 'Region',

       'Metro/Regional', 'Gaming'] #pick the ones that are useful

In [16]:
#categories

pd.unique(locations['Category'])

on_license = ['General Licence', 'Late night (general) Licence', 'Full Club Licence', 'Late night (on-premises) Licence', 'On-Premises Licence']

off_license = ['General Licence', 'Late night (general) Licence', 'Late night (packaged liquor) Licence', 'Late night (general) Licence', 'Packaged Liquor Licence']

In [17]:
#all locations

all_locationsXY = locations[select_cols]

all_locationsXY.to_csv("alcohol_locations.csv")

all_locationsXY.head()

,Trading As,Category,Trading Hours,Address,Suburb,Postcode,Latitude,Longitude,Postal Postcode,Council,Region,Metro/Regional,Gaming
0,RESERVOIR CRICKET CLUB,Limited Licence,Limited Licence,DONATH RESERVE WEST PAVILION JOHNSON STREET,RESERVOIR,3073.0,-37.699197,145.011408,3073,DAREBIN CITY COUNCIL,Metro North,Metro,N
1,WODONGA CRICKET CLUB,Limited Licence,Limited Licence,GORDON STREET,WODONGA,3690.0,-36.134678,146.882671,3690,WODONGA CITY COUNCIL,Hume,Regional,N
2,HAPPY WINE WOMAN,Limited Licence,Limited Licence,U 209 12-14 DICKENS ST,ELWOOD,3184.0,-37.871375,144.986657,3184,PORT PHILLIP CITY COUNCIL,Metro South,Metro,N
3,TACO BILL MEXICAN CANTINA,Limited Licence,Limited Licence,GROUND FLOOR 375-381 CLARENDON STREET,SOUTH MELBOURNE,3205.0,-37.836147,144.962283,3171,PORT PHILLIP CITY COUNCIL,Metro South,Metro,N
4,LANEWAY GREENS,Limited Licence,Limited Licence,"LEVEL 13, 564 ST KILDA ROAD",MELBOURNE SOUTH,3004.0,-37.846584,144.978855,3004,PORT PHILLIP CITY COUNCIL,Metro South,Metro,N


In [18]:
#on license

on_licenseXY = locations.loc[locations['Category'].isin(on_license)]

pd.unique(on_licenseXY['Category'])

on_licenseXY.to_csv("on_license_locations.csv")

In [19]:
#off license

off_licenseXY = locations.loc[locations['Category'].isin(off_license)]

pd.unique(off_licenseXY['Category'])

off_licenseXY.to_csv("off_license_locations.csv")

In [20]:
# for the map
off_licenseXY_metro = off_licenseXY.loc[off_licenseXY.Latitude.notnull()] #drops the NAs
off_licenseXY_metro = off_licenseXY_metro[off_licenseXY_metro['Region'] == "Metro North"] # Metro North only - folium (in jupyter doesn't like too many rows it seems)

In [21]:
#from https://nbviewer.jupyter.org/gist/ocefpaf/d1d612d290f766935d8abe1559523a72

location = off_licenseXY_metro['Latitude'].mean(), off_licenseXY['Longitude'].mean()

locationlist = off_licenseXY_metro[["Latitude","Longitude"]].values.tolist()
labels = off_licenseXY_metro["Trading As"].values.tolist()

m = folium.Map(location=location, zoom_start=14)
for point in range(len(locationlist)):
    popup = folium.Popup(labels[point], parse_html=True)
    folium.Marker(locationlist[point], popup=popup).add_to(m)

m